In [1]:
!pip install fasttext
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')
ft = fasttext.load_model('cc.en.300.bin')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 1.9 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3158069 sha256=aacaa63b91614ba220e01fed7be27c6e7672c3cfec13f511c387c09d1cbc838b
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext



In [2]:
import pandas as pd
import numpy as np
# data = pd.read_csv("Processed.csv")
data = pd.read_csv("balanced.csv")
df = data[['Content','SourceatBot7']]
df = df[df['SourceatBot7'].str.contains("Human") | df['SourceatBot7'].str.contains("Bot")]

In [3]:
def generateVector(sentence):
    return ft.get_sentence_vector(sentence)

feature = df['Content'].apply(generateVector)   

In [4]:
feature

0        [-0.0028227102, 0.011637921, -0.023345655, 0.0...
1        [0.011710363, 0.007031395, 0.0024293398, 0.054...
2        [-0.015865443, -0.031779964, 0.00086557923, 0....
3        [-0.02534811, -0.008070641, -0.026338749, 0.06...
4        [-0.023846624, -0.005181138, -0.008818034, 0.0...
                               ...                        
53417    [-0.028007094, -0.011354605, 0.003258219, 0.08...
53418    [-0.033677645, -0.02218605, -0.045550987, 0.03...
53419    [-0.006191723, -0.00435643, -0.012524227, 0.05...
53420    [-0.028489707, -0.0118733505, -0.0276893, 0.05...
53421    [-0.0037955444, -0.0013152966, -0.02190763, 0....
Name: Content, Length: 53422, dtype: object

In [5]:
max_len = max([len(x) for x in feature])
padded = [np.pad(x, (0, max_len - len(x)), 'constant') for x in feature]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
labels = df['SourceatBot7'] == "Human"
labels = labels * 1
# X_train, X_test, y_train, y_test = train_test_split(feature, labels, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=1)

In [7]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
print('LGR Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('LGR Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print('LGR Testing CM: {}\n'.format(confusion_matrix(y_test, y_pred)))

LGR Testing accuracy 0.6191857744501638
LGR Testing F1 score: 0.6191316024986294
LGR Testing CM: [[3268 2136]
 [1933 3348]]



In [9]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [10]:
print('NB Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('NB Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print('NB Testing CM: {}\n'.format(confusion_matrix(y_test, y_pred)))

NB Testing accuracy 0.5925128685072532
NB Testing F1 score: 0.5909669618735652
NB Testing CM: [[3526 1878]
 [2476 2805]]



In [11]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [12]:
print('SVM Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('SVM Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))
print('SVM Testing CM: {}\n'.format(confusion_matrix(y_test, y_pred)))

SVM Testing accuracy 0.6160973327094057
SVM Testing F1 score: 0.6160760876727889
SVM Testing CM: [[3272 2132]
 [1970 3311]]

